# Syntax of PartiQL

The syntax of PartiQL is completely provisional, but there are reasons behind each choice. In this notebook, I will explain those reasons.

## Why not SQL?

Although I want to demonstrate the value of relational concepts like surrogate indexes and set operations for particle physics, strict SQL would limit usability in ways that would be too distracting for the demo.

   * PartiQL indexes are not visible, but SQL's are. In PartiQL, we are only using the indexes and index-matching to ensure that particles retain their identity, so there is only one choice for the `ON` clause of a `JOIN`. SQL is more general: SQL users sometimes want to match on surrogate keys, sometimes natural keys, and their choice will depend on their domain. Following this path, PartiQL should at least drop SQL's `ON` clause.
   * SQL's {database, table, row} hierarchy corresponds to the awkward array structure `ListArray(RecordArray(PrimitiveArrays...)))`. In particle physics, we want to deal with more structures than this. It would be possible in SQL by emulating deeper structures using table normalization, but that would require an `ON` clause to select the right foreign keys to link tables. Arguably, what PartiQL does is internally manage foreign keys with its implicit `ON` clauses to provide the appearance of deeply nested data structures, which makes it more high-level than SQL: it maintains data in a way that is appropriate for particle physics only.
   * If we apply queries to individual events, we will need new constructs to perform operations across events, such as cutting events and histogramming.
   * SQL seems to have bad design decisions, patched over by decades of practice (e.g. [common table expressions](https://www.citusdata.com/blog/2018/08/09/fun-with-sql-common-table-expressions/) is a heavy boilerplate and out-of-order way to do functional composition, the evaluation order is visible and [very different](https://sqlbolt.com/lesson/select_queries_order_of_execution) from the order in which queries are written, etc.). Adhering to SQL's syntax would put a burden on physicists who are new to it.
   * PartiQL queries are never going to be exact SQL queries, so the value of interoperability is at the level of concepts: for that, we use the same names. Data scientists who know SQL will find familiar ideas in PartiQL and physicists who learn something like PartiQL will recognize those terms when they encounter them in SQL.
   * The `cut/vary/hist` block syntax of my October 2018 language was a good idea and should be replicated here.

## General design

PartiQL has an expression syntax like Python's, including Python's operator precedence. It has the standard binary operators `+`, `-`, `*`, `/`, and `**`, comparison operators (not chained as in Python, but that would be a good idea), and `==` checks for equality. The logical operators are words: `and`, `or`, `not`, and there are no bitwise operators. Use `in` or `not in` to determine whether a value is a member of a set.

Python and C/C++ comments are both supported.

As we will see in the notebook about runtime evaluation, missing data are supported but not explicitly visible to the user as a `None` or `null` object. (A set containing missing data is the same as an empty set and a missing value for a record field is the same as not having that field.) To support missing data handling, PartiQL has [safe navigation operators](https://en.wikipedia.org/wiki/Safe_navigation_operator) `?possibly_missing` and `not_missing?.possibly_missing`, as well as a `has possibly_missing` expression that returns true or false.

Join operations are words, `join`, `cross`, `union`, `except`, and are infix binary operators, just as they are in SQL. However, they form part of the general expression syntax with other binary operators (e.g. `+`, `-`, `*`, `/`), though with lower precedence. Similarly, `group by` and `min by`/`max by` are binary operators.

Curly brackets are used (1) to isolate scope, so that temporary variables do not overshadow other variables with the same name, (2) to add data to a set of records after a `with` keyword, and (3) to enclose nested `cut/vary/hist` statements.

Whitespace is not significant, except that a newline can end a statement (as can a semicolon `;`). Nesting is expressed with curly brackets `{...}`, not indentation.

The outermost structures can be `cut` (to select events) and/or `vary` (to compute a block with different inputs for systematic variations). These statements are like the `region` and `vary` of the October 2018 language, but not as extensive and interchangeable with histogram binning specifiers. (It would be good to make them so.) Like the October 2018 language, `cut` and `vary` can be nested within each other, and doing so builds a hierarchy of counters for cut flows. Events have weights, which are multiplied by expressions in `weight by ...` clauses as an event descends the hierarchy.

Histograms can be placed anywhere in the hierarchy with a `hist` statement, and the corresponding histogram appears in the hierarchy of counters. This way, the binning, name, and title of the histogram are specified next to the expressions to fill and `weight by`. The number of entries in a histogram depends on its location in the hierarchy of cuts and whether it is in any `with` expressions for a set of records.

Although the language has no functions, macros (defined with `def`) are expanded during parsing to avoid repetitive typing. Macros are not runtime objects—it is as though all functions are inlined.

## Illustrative examples

To execute the examples, be sure to have the [Lark parser](https://github.com/lark-parser/lark#readme) installed.

In [1]:
!pip install lark-parser

In [2]:
import parser